In [19]:
import pandas as pd
import error_prompts as p
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()



prompt_name = 'confidence'
prompt = p.confidence

file = "singleterm_dynamic_fewshot_4-turbo-preview"
MODEL = "gpt-3.5-turbo"

DATA_PATH = f'../../Datasets/Evaluations/Sentiment Analysis/{file}.csv'
OUTPUT_PATH = f"../../Datasets/Evaluations/Sentiment Analysis/Error_Analysis/{prompt_name}_{file}_with_{MODEL}.csv"
TEMP = 0





In [20]:
df = pd.read_csv(DATA_PATH)
print(df.head())

   Unnamed: 0.1  Unnamed: 0  text_id  term_id  \
0             0           0        0        0   
1             1           1        0        1   
2             2           2        0        2   
3             3           3        0        3   
4             4           4        1        4   

                                                text  \
0  Amusing details distinguish desserts, from dul...   
1  Amusing details distinguish desserts, from dul...   
2  Amusing details distinguish desserts, from dul...   
3  Amusing details distinguish desserts, from dul...   
4  The server was so busy the night we visited th...   

                          term  polarity  example1_term_id  \
0                     desserts  positive              9500   
1     dulce de leche ice-cream   neutral              9500   
2  chocolate sauce tic-tac-toe   neutral              9500   
3            poached pineapple   neutral              9500   
4                       server  negative              1211

In [21]:
df['analysis'] = None


In [22]:
def execute(df, prompt):
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=10)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=10)
    for i in range(len(df)):
        if(df["analysis"][i] != None and df["analysis"][i] != "error"):
            continue
        print(i)
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        user_prompt = df.loc[i, 'prompt']
        ai_answer = df.loc[i, 'polarity_pred']
        try: 
            result = chain.run(user_prompt = user_prompt, ai_answer = ai_answer, callbacks=[handler])
        except Exception as e:
            print(e)
            print(chain.prompt.format_prompt(user_prompt = user_prompt, ai_answer = ai_answer).to_string())
            df['analysis'][i] = 'error'
            continue
        handler.langfuse.flush()
        prompt_text = chain.prompt.format_prompt(user_prompt = user_prompt, ai_answer = ai_answer).to_string()
        if (i < 5):
            print(prompt_text)
            print("\n")
            print(result)
            print("\n")
        
        if (i % 50 == 0):
            print(str(i) + " of  " + str(len(df)))
        df.loc[i, 'analysis']= result
    return df


df = execute(df, prompt)
    



0
System: You are a helpful AI.
Human: Example 1:
Input: "Food is mediocre, except for Mushroom Custard and Foie Gras in Fire Ice."
Term: Food
Output: negative


Example 2:
Input: "It was the WORST Sangria ever, and their house drink has just a shot of Malibu rum, and at a price of $9 you want more than a shot of a 40% alcohol."
Term: Sangria
Output: negative


Example 3
Input: "The overpriced ice cream is delicious but the service is atrocious - not only do I have to wait to be admitted into this ice cream joint, but the waiter is a snide, gruff man more suited to be a trucker than an ice cream shop server."
Term: service
Output: negative

    
Task:
Input: Amusing details distinguish desserts, from dulce de leche ice-cream profiteroles dotting a chocolate sauce tic-tac-toe board, to coconut custard surrounded by a sea of Malibu-rum gelee and poached pineapple.
Prompt: What is the sentiment in the text towards 'desserts'? Only respond with "positive", "negative" or "neutral" as one wo

In [23]:
print(df['analysis'])

0      Confidence: 90%
1      Confidence: 90%
2      Confidence: 90%
3      Confidence: 90%
4      Confidence: 90%
            ...       
446    Confidence: 90%
447    Confidence: 80%
448    Confidence: 80%
449    Confidence: 90%
450    Confidence: 70%
Name: analysis, Length: 451, dtype: object


In [18]:
df.to_csv(OUTPUT_PATH, index=False)
